## 📦 Instalación de Dependencias

In [ ]:
!pip install -q flask flask-ngrok pyngrok pandas numpy scikit-learn joblib

## 🔧 Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib
import json
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading

## 🎯 Configurar Token de Ngrok

**IMPORTANTE:** Obtén tu token de ngrok en https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# ⚠️ REEMPLAZA CON TU TOKEN DE NGROK
NGROK_AUTH_TOKEN = "tu_token_aqui"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## 🧠 Clase del Modelo ML

In [ ]:
class ExpensePredictionModel:
    def __init__(self):
        self.model = None
        self.anomaly_detector = None
        self.label_encoders = {}
        self.feature_names = []
        self.metrics = {}
        self.is_trained = False
        
    def prepare_features(self, data):
        """Preparar features para el modelo"""
        df = pd.DataFrame(data)
        
        # Convertir fecha a datetime
        df['fecha'] = pd.to_datetime(df['fecha'])
        
        # Extraer features temporales
        df['day_of_week'] = df['fecha'].dt.dayofweek
        df['day_of_month'] = df['fecha'].dt.day
        df['month'] = df['fecha'].dt.month
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        df['week_of_month'] = (df['day_of_month'] - 1) // 7 + 1
        
        # Codificar categorías
        if 'categoria' in df.columns:
            if 'categoria' not in self.label_encoders:
                self.label_encoders['categoria'] = LabelEncoder()
                df['categoria_encoded'] = self.label_encoders['categoria'].fit_transform(df['categoria'])
            else:
                df['categoria_encoded'] = self.label_encoders['categoria'].transform(df['categoria'])
        
        # Features de usuario
        if 'userId' in df.columns:
            if 'userId' not in self.label_encoders:
                self.label_encoders['userId'] = LabelEncoder()
                df['user_encoded'] = self.label_encoders['userId'].fit_transform(df['userId'])
            else:
                df['user_encoded'] = self.label_encoders['userId'].transform(df['userId'])
        
        return df
    
    def train(self, training_data, model_config):
        """Entrenar el modelo con los datos proporcionados"""
        print(f"🚀 Iniciando entrenamiento con {len(training_data)} registros...")
        
        # Preparar datos
        df = self.prepare_features(training_data)
        
        # Seleccionar features
        feature_columns = ['day_of_week', 'day_of_month', 'month', 'is_weekend', 
                          'week_of_month', 'categoria_encoded']
        
        if 'user_encoded' in df.columns:
            feature_columns.append('user_encoded')
        
        X = df[feature_columns]
        y = df['monto']
        
        self.feature_names = feature_columns
        
        # Dividir datos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Entrenar modelo
        n_estimators = model_config.get('n_estimators', 100)
        max_depth = model_config.get('max_depth', 10)
        
        self.model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )
        
        self.model.fit(X_train, y_train)
        
        # Evaluar
        y_pred = self.model.predict(X_test)
        
        self.metrics = {
            'mae': float(mean_absolute_error(y_test, y_pred)),
            'rmse': float(np.sqrt(mean_squared_error(y_test, y_pred))),
            'r2': float(r2_score(y_test, y_pred)),
            'accuracy': float(r2_score(y_test, y_pred) * 100),
            'training_samples': len(X_train),
            'test_samples': len(X_test)
        }
        
        # Entrenar detector de anomalías
        self.anomaly_detector = IsolationForest(
            contamination=0.1,
            random_state=42
        )
        self.anomaly_detector.fit(X)
        
        self.is_trained = True
        
        print(f"✅ Modelo entrenado exitosamente!")
        print(f"📊 R² Score: {self.metrics['r2']:.4f}")
        print(f"📉 MAE: ${self.metrics['mae']:.2f}")
        print(f"📈 RMSE: ${self.metrics['rmse']:.2f}")
        
        return self.metrics
    
    def predict(self, user_data, days_to_predict=30):
        """Hacer predicciones para un usuario"""
        if not self.is_trained:
            raise Exception("Modelo no entrenado. Llama a train() primero.")
        
        # Preparar features
        df = self.prepare_features(user_data)
        
        # Generar fechas futuras
        last_date = df['fecha'].max()
        future_dates = [last_date + timedelta(days=i) for i in range(1, days_to_predict + 1)]
        
        # Crear dataframe de predicción
        predictions = []
        
        for date in future_dates:
            pred_row = {
                'day_of_week': date.weekday(),
                'day_of_month': date.day,
                'month': date.month,
                'is_weekend': int(date.weekday() >= 5),
                'week_of_month': (date.day - 1) // 7 + 1,
                'categoria_encoded': df['categoria_encoded'].mode()[0] if 'categoria_encoded' in df.columns else 0
            }
            
            if 'user_encoded' in df.columns:
                pred_row['user_encoded'] = df['user_encoded'].iloc[0]
            
            pred_df = pd.DataFrame([pred_row])[self.feature_names]
            predicted_amount = self.model.predict(pred_df)[0]
            
            predictions.append({
                'date': date.strftime('%Y-%m-%d'),
                'predicted_amount': float(predicted_amount)
            })
        
        # Calcular estadísticas
        total_predicted = sum(p['predicted_amount'] for p in predictions)
        avg_predicted = total_predicted / len(predictions)
        
        # Calcular confianza basada en el R²
        confidence = self.metrics.get('r2', 0.5) * 100
        
        return {
            'predictions': predictions,
            'total_predicted': float(total_predicted),
            'average_daily': float(avg_predicted),
            'confidence': float(confidence),
            'days': days_to_predict
        }
    
    def detect_anomalies(self, expenses):
        """Detectar gastos anómalos"""
        if not self.is_trained or self.anomaly_detector is None:
            raise Exception("Modelo no entrenado")
        
        df = self.prepare_features(expenses)
        X = df[self.feature_names]
        
        # Predecir anomalías (-1 = anomalía, 1 = normal)
        predictions = self.anomaly_detector.predict(X)
        anomaly_scores = self.anomaly_detector.score_samples(X)
        
        anomalies = []
        for idx, (pred, score) in enumerate(zip(predictions, anomaly_scores)):
            if pred == -1:
                anomalies.append({
                    'index': int(idx),
                    'amount': float(df.iloc[idx]['monto']),
                    'category': df.iloc[idx].get('categoria', 'Unknown'),
                    'date': df.iloc[idx]['fecha'].strftime('%Y-%m-%d'),
                    'anomaly_score': float(score),
                    'severity': 'high' if score < -0.5 else 'medium'
                })
        
        return {
            'anomalies_found': len(anomalies),
            'anomalies': anomalies,
            'total_analyzed': len(expenses)
        }

# Instancia global del modelo
model = ExpensePredictionModel()

## 🌐 API Flask

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health_check():
    """Verificar estado de la API"""
    return jsonify({
        'status': 'healthy',
        'model_trained': model.is_trained,
        'timestamp': datetime.now().isoformat()
    }), 200

@app.route('/train', methods=['POST'])
def train_model():
    """Entrenar el modelo"""
    try:
        data = request.get_json()
        training_data = data.get('training_data', [])
        model_config = data.get('model_config', {})
        
        if not training_data:
            return jsonify({'error': 'No training data provided'}), 400
        
        metrics = model.train(training_data, model_config)
        
        return jsonify({
            'status': 'success',
            'message': 'Model trained successfully',
            'metrics': metrics,
            'accuracy': metrics['accuracy']
        }), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/predict', methods=['POST'])
def predict():
    """Hacer predicción para un usuario"""
    try:
        data = request.get_json()
        user_id = data.get('user_id')
        historical_data = data.get('historical_data', [])
        days_to_predict = data.get('days_to_predict', 30)
        
        if not historical_data:
            return jsonify({'error': 'No historical data provided'}), 400
        
        result = model.predict(historical_data, days_to_predict)
        result['user_id'] = user_id
        result['predicted_amount'] = result['total_predicted']
        result['confidence'] = result['confidence']
        
        return jsonify(result), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/predict_multiple', methods=['POST'])
def predict_multiple():
    """Predicciones para múltiples categorías"""
    try:
        data = request.get_json()
        user_id = data.get('user_id')
        historical_data = data.get('historical_data', [])
        categories = data.get('categories', [])
        days_to_predict = data.get('days_to_predict', 30)
        
        predictions_by_category = {}
        
        for category in categories:
            category_data = [d for d in historical_data if d.get('categoria') == category]
            if category_data:
                pred = model.predict(category_data, days_to_predict)
                predictions_by_category[category] = pred
        
        return jsonify({
            'user_id': user_id,
            'predictions': predictions_by_category
        }), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/detect_anomalies', methods=['POST'])
def detect_anomalies():
    """Detectar gastos anómalos"""
    try:
        data = request.get_json()
        expenses = data.get('expenses', [])
        
        if not expenses:
            return jsonify({'error': 'No expenses provided'}), 400
        
        result = model.detect_anomalies(expenses)
        
        return jsonify(result), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/metrics', methods=['GET'])
def get_metrics():
    """Obtener métricas del modelo"""
    if not model.is_trained:
        return jsonify({'error': 'Model not trained yet'}), 400
    
    return jsonify(model.metrics), 200

@app.route('/recommendations', methods=['POST'])
def get_recommendations():
    """Obtener recomendaciones personalizadas"""
    try:
        data = request.get_json()
        user_data = data.get('user_data', {})
        
        # Análisis básico para recomendaciones
        recommendations = [
            {
                'type': 'budget',
                'message': 'Basado en tus patrones de gasto, considera ajustar tu presupuesto',
                'priority': 'medium'
            },
            {
                'type': 'savings',
                'message': 'Puedes ahorrar más reduciendo gastos en categorías no esenciales',
                'priority': 'high'
            }
        ]
        
        return jsonify({
            'recommendations': recommendations,
            'generated_at': datetime.now().isoformat()
        }), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/feedback', methods=['POST'])
def receive_feedback():
    """Recibir feedback sobre predicciones"""
    try:
        data = request.get_json()
        # Guardar feedback para mejorar el modelo (implementar persistencia)
        print(f"📝 Feedback recibido: {data}")
        
        return jsonify({'status': 'feedback_received'}), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

print("✅ API Flask configurada")

## 🚀 Iniciar Servidor con Ngrok

In [ ]:
# Iniciar servidor en un thread
def run_app():
    app.run(port=5000)

# Iniciar Flask en background
thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

print("⏳ Esperando a que Flask inicie...")
import time
time.sleep(3)

# Crear túnel ngrok
public_url = ngrok.connect(5000, bind_tls=True)
print("\n" + "="*60)
print("🎉 ¡SERVIDOR INICIADO EXITOSAMENTE!")
print("="*60)
print(f"\n📡 URL Pública de la API: {public_url}")
print("\n⚠️ IMPORTANTE: Copia esta URL y úsala en tu app Flutter:")
print(f"\n   colabService.setColabApiUrl('{public_url}');")
print("\n" + "="*60)
print("\n✅ Endpoints disponibles:")
print(f"   • Health Check: {public_url}/health")
print(f"   • Entrenar: {public_url}/train")
print(f"   • Predecir: {public_url}/predict")
print(f"   • Múltiples: {public_url}/predict_multiple")
print(f"   • Anomalías: {public_url}/detect_anomalies")
print(f"   • Métricas: {public_url}/metrics")
print(f"   • Recomendaciones: {public_url}/recommendations")
print("\n" + "="*60)
print("\n💡 El servidor se mantendrá activo mientras esta celda esté ejecutándose")
print("   NO cierres este notebook ni detengas la ejecución\n")

## 🧪 Prueba la API

In [ ]:
import requests

# Test health check
response = requests.get(f"{public_url}/health")
print("🔍 Health Check:")
print(json.dumps(response.json(), indent=2))

## 📊 Ejemplo de Entrenamiento

In [ ]:
# Datos de ejemplo para entrenar
sample_data = [
    {'userId': 'user1', 'fecha': '2024-01-01', 'monto': 50.0, 'categoria': 'Comida'},
    {'userId': 'user1', 'fecha': '2024-01-02', 'monto': 30.0, 'categoria': 'Transporte'},
    {'userId': 'user1', 'fecha': '2024-01-03', 'monto': 100.0, 'categoria': 'Comida'},
    {'userId': 'user2', 'fecha': '2024-01-01', 'monto': 75.0, 'categoria': 'Entretenimiento'},
    {'userId': 'user2', 'fecha': '2024-01-02', 'monto': 45.0, 'categoria': 'Comida'},
]

# Entrenar modelo
train_response = requests.post(
    f"{public_url}/train",
    json={
        'training_data': sample_data,
        'model_config': {'n_estimators': 100, 'max_depth': 10}
    }
)

print("\n🎓 Resultado del entrenamiento:")
print(json.dumps(train_response.json(), indent=2))

## 🔮 Ejemplo de Predicción

In [ ]:
# Hacer predicción
predict_response = requests.post(
    f"{public_url}/predict",
    json={
        'user_id': 'user1',
        'historical_data': sample_data[:3],
        'days_to_predict': 7
    }
)

print("\n🔮 Predicción:")
print(json.dumps(predict_response.json(), indent=2))

## 💾 Guardar Modelo (Opcional)

In [ ]:
# Guardar modelo entrenado
if model.is_trained:
    joblib.dump(model.model, 'expense_prediction_model.pkl')
    joblib.dump(model.anomaly_detector, 'anomaly_detector.pkl')
    joblib.dump(model.label_encoders, 'label_encoders.pkl')
    
    print("✅ Modelo guardado exitosamente")
    print("   • expense_prediction_model.pkl")
    print("   • anomaly_detector.pkl")
    print("   • label_encoders.pkl")
else:
    print("⚠️ El modelo no está entrenado aún")

## 📥 Cargar Modelo Guardado (Opcional)

In [ ]:
# Cargar modelo previamente guardado
try:
    model.model = joblib.load('expense_prediction_model.pkl')
    model.anomaly_detector = joblib.load('anomaly_detector.pkl')
    model.label_encoders = joblib.load('label_encoders.pkl')
    model.is_trained = True
    
    print("✅ Modelo cargado exitosamente")
except FileNotFoundError:
    print("⚠️ No se encontraron archivos de modelo guardados")